# Setup

In [ ]:
# OctoAI
# ! pip install langchain langchain-community faiss-cpu sentence-transformers octoai langchain-text-splitters lxml tiktoken python-dotenv

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()
OCTOAI_API_TOKEN = os.environ["OCTOAI_API_TOKEN"]

# Ingest Data

In [2]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.schema import Document

In [4]:
files = os.listdir("../../city_data")
file_texts = []
for file in files:
    with open(f"../../city_data/{file}") as f:
        file_text = f.read()
    text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
        chunk_size=512, chunk_overlap=64, 
    )
    texts = text_splitter.split_text(file_text)
    for i, chunked_text in enumerate(texts):
        file_texts.append(Document(page_content=chunked_text, 
                metadata={"doc_title": file.split(".")[0], "chunk_num": i}))

Created a chunk of size 1311, which is longer than the specified 512
Created a chunk of size 536, which is longer than the specified 512
Created a chunk of size 676, which is longer than the specified 512
Created a chunk of size 745, which is longer than the specified 512
Created a chunk of size 558, which is longer than the specified 512
Created a chunk of size 671, which is longer than the specified 512
Created a chunk of size 631, which is longer than the specified 512
Created a chunk of size 704, which is longer than the specified 512
Created a chunk of size 528, which is longer than the specified 512
Created a chunk of size 765, which is longer than the specified 512
Created a chunk of size 527, which is longer than the specified 512
Created a chunk of size 635, which is longer than the specified 512
Created a chunk of size 618, which is longer than the specified 512
Created a chunk of size 614, which is longer than the specified 512
Created a chunk of size 666, which is longer th

In [5]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

In [6]:
embeddings = HuggingFaceEmbeddings()

/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be r

In [7]:
vector_store = FAISS.from_documents(
    file_texts,
    embedding=embeddings
)

# Search the Data

In [8]:
from langchain_community.llms.octoai_endpoint import OctoAIEndpoint
llm = OctoAIEndpoint(
        model="meta-llama-3-8b-instruct",
        max_tokens=1024,
        presence_penalty=0,
        temperature=0.1,
        top_p=0.9,
    )

/Users/yujian/Documents/workspace/rag_cookbooks/rcbs/lib/python3.12/site-packages/langchain_core/utils/utils.py:161: UserWarning: WARNING! model is not default parameter.
                model was transferred to model_kwargs.
                Please confirm that model is what you intended.
  warnings.warn(


In [9]:
retriever = vector_store.as_retriever()

In [10]:
from langchain.prompts import ChatPromptTemplate
template="""You are a tour guide. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

In [11]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [16]:
chain.invoke("What is a good landmark to visit in Paris?")

' The Eiffel Tower is a good landmark to visit in Paris. It is one of the most iconic and recognizable landmarks in the world and offers stunning views of the city. It is also a popular spot for tourists and locals alike. \nContext: [Document(page_content=\'== Cityscape ==\\n\\n\\n=== Urbanism and architecture ===\\n\\nParis is one of the few world capitals that has rarely seen destruction by catastrophe or war. For this, even its earliest history is still visible in its streetmap, and centuries of rulers adding their respective architectural marks on the capital has resulted in an accumulated wealth of history-rich monuments and buildings whose beauty played a large part in giving the city the reputation it has today. At its origin, before the Middle Ages, the city was composed of several islands and sandbanks in a bend of the Seine; of those, two remain today: Île Saint-Louis and the Île de la Cité. A third one is the 1827 artificially created Île aux Cygnes.\\nModern Paris owes much